In [ ]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3

import matplotlib as mpl
import matplotlib.pyplot as plt

# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import cv2
import pims
import trackpy as tp
import glob
import os
import os.path
import seaborn as sns
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

In [ ]:
frames = pims.open('C:/Users/Crazz/Research Codes/GoPro extracted images/*.png')
tp.quiet()
f = tp.batch(frames[:], 11, invert=False)
t = tp.link(f, 15, memory=3)
t = tp.filter_stubs(t, 2)
v_allp = []
size_allp = []
partnum = []
countvars = 0
countsplits = 0
for i in t.particle.unique():
    p_data = t[t.particle==i]
    dy = np.diff(p_data.y)
    dx = np.diff(p_data.x)
    frame_diff = np.diff(p_data.frame)
    dy[abs(dx)>4] = 0
    dy[dy<0] = 0
    dy = dy/frame_diff
    v = dy*119.90*0.0624/1000*10 # image one tenth of original size, original scale 0.0624 mm/pixel. 119.9 fps
    size = p_data['size']
    part = i
    if len(np.where(v==0)[0])!= 0:
        vsplit = np.array_split(v, np.where(v==0)[0]+1)
        for j in range(len(vsplit)):
            vsplit[j][vsplit[j]==0] = np.nan
            if np.nanvar(vsplit[j]) > 0.01:
                vsplit[j] = np.nan
                countvars+=1
            size_allp.append(0)
            partnum.append(i)
            countsplits+=1
            v_allp.append(np.nanmean(vsplit[j],axis=0))
            
    else:
        if np.nanvar(v) > 0.01:
            v = np.nan
            size = np.nan
        v_allp.append(np.nanmean(v))
        size_allp.append(np.nanmean(size))
        partnum.append(part)
v_allp = np.array(v_allp)
size_allp = np.array(size_allp)
partnum = np.array(partnum)
size_allp = size_allp[~np.isnan(v_allp)]
partnum = partnum[~np.isnan(v_allp)]
v_allp = v_allp[~np.isnan(v_allp)]
part_data = pd.DataFrame(columns=['v_ms','rg_mm','part'])
size_allmm = size_allp*0.0624*10
part_data['v_ms']=v_allp
part_data['rg_mm']=size_allmm
part_data['part']=partnum
part_data.to_csv('C:/Users/Crazz/Research Codes/Snowflake PTV/1_7_1129_10sec_dat.csv')